# Homework Batch III: Routing Algorithms
## Davide Basso - SM3500450

1. Implement the binary heap-based version of the Dijkstra's algorithm.

First of all I've done some slight modifications on the binheap class that we've implemented during lesson in order to properly deal with nodes. These can be found in the file **node_binheap.py** present in this repository; basically since we are considering distances in order to build the heap, all the comparison steps were done with respect to distances and not node's index. Further insights can be found as comments in the same file.

Then I've implemented `Node`, `Edge` and `Weighted_Graph` classes, within **graph.py**, that are strictly necessary in order to implement Dijkstra's Algorithm. Also in this case I recall comments present in the aforementioned file, in order to grasp all the properties of each object.




In [1]:
from math import inf
from graph import *
from node_binheap import binheap

In [2]:
def init_SSSP(G):
    for v in G.V:
        v.d = inf
        v.pred = None

def relax(Q, u, v, w):
    if u.d+w < v.d:
        Q.decrease_key(v.heap_index, u.d+w)
        v.pred = u

def Dijkstra(G, s):
    init_SSSP(G)
    s.d = 0

    Q = binheap(G.V)
    for h_i, node in enumerate(Q._A):
        node.heap_index = h_i

    final_queue = []
    i = 0
    while not Q.is_empty():
        u = Q.remove_minimum()
        final_queue.append(u)

        # this means that we've arrived at the point of the queue
        # in which are located only nodes eliminated by shortcuts
        if u.d == inf and u.pred is None:
            break

        if i != 0:
            print(f"\nnode {u.index} with total distance {u.d} from root and predecessor node is {u.pred.index}")
        else:
            i += 1
            print(f"The path starts from node {u.index}, then we have:")

        for (v,w) in G.Adj[u.index]:
            relax(Q, u, v, w)
    
    return final_queue

In [3]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,1), Edge(a,c,5), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]

In [4]:
G = Weighted_Graph(vertices, edges)
for v in G.V:
    v.print_node()

index: 0, data: None, predecessor: None
index: 1, data: None, predecessor: None
index: 2, data: None, predecessor: None
index: 3, data: None, predecessor: None
index: 4, data: None, predecessor: None
index: 5, data: None, predecessor: None


In [5]:
res = Dijkstra(G,a)

The path starts from node 0, then we have:

node 1 with total distance 1 from root and predecessor node is 0

node 2 with total distance 5 from root and predecessor node is 0

node 3 with total distance 7 from root and predecessor node is 2

node 4 with total distance 8 from root and predecessor node is 3

node 5 with total distance 11 from root and predecessor node is 4


2. Consider the contraction hierarchies presented during the course. Assume to deal with graphs that can be fully represented in the memory of your computer. Implement:

(a) an algorithm to add the shortcuts to a graph;


In [6]:
def compute_shortcut(node, edges, vertices):
    shortcuts = {"deleted": None,
                 "pred": [], "pred_weight":[],
                 "next": [], "next_weight": []}

    shortcuts["deleted"] = node.index

    # find the edges in which node is involved
    # and add them to the dictionary
    for edge in edges:
        if edge.v.index == node.index:
            shortcuts["pred"].append(edge.u.index)
            shortcuts["pred_weight"].append(edge.weight)
        if edge.u.index == node.index:
            shortcuts["next"].append(edge.v.index)
            shortcuts["next_weight"].append(edge.weight)

    for k,v in shortcuts.items():
        print(f"{k} -> {v}")
    
    # remove edges involving node
    res = [edge for edge in edges if not (edge.v.index == node.index or edge.u.index == node.index)]
    
    # create new edges
    new_edges = []
    for pred, pred_w in zip(shortcuts["pred"], shortcuts["pred_weight"]):
        #print(pred,"and", pred_w)
        p = [v for v in vertices if v.index == pred][0]
        for succ, succ_w in zip(shortcuts["next"], shortcuts["next_weight"]):
            #print(succ,"and", succ_w)
            s = [v for v in vertices if v.index == succ][0]
            new_edge = Edge(p, s, pred_w+succ_w)
            new_edges.append(new_edge)
            res.append(new_edge)
    
    # compare if new edges are worse than already existing ones
    # if so remove them
    for n_e in new_edges:
        for e in edges:
            if e.u.index == n_e.u.index and e.v.index == n_e.v.index and e.weight < n_e.weight:
                res.remove(n_e)
                
    return res

In [7]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,10), Edge(a,c,1), Edge(b,c,3), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]

In [8]:
res = compute_shortcut(a, edges, vertices)

print("\n")
for el in res:
    print(f"pred: {el.u.index}, succ: {el.v.index}, val: {el.weight}")

deleted -> 0
pred -> []
pred_weight -> []
next -> [1, 2]
next_weight -> [10, 1]


pred: 1, succ: 2, val: 3
pred: 1, succ: 5, val: 15
pred: 2, succ: 3, val: 2
pred: 3, succ: 4, val: 1
pred: 4, succ: 5, val: 3


(b) a bidirectional version of Dijkstra algorithm that can operate on the graphs decorated by the algorithm at Point 2a.

In [9]:
def reverse_edges(edges):
    back = []
    for el in edges:

        tmp = Edge(el.v, el.u, el.weight)
        back.append(tmp)

    return back

In [10]:
def get_predecessors(node, predecs):
    if node.pred is not None:
        predecs.append(node.pred)
        get_predecessors(node.pred, predecs)

In [11]:
from copy import deepcopy
def bidirect_Dijkstra(f_G, b_G, s, t):
    F_G = deepcopy(f_G)
    B_G = deepcopy(b_G)
    init_SSSP(F_G)
    init_SSSP(B_G)

    F_G.V[s.index].d = 0
    f_Q = binheap(F_G.V)

    B_G.V[t.index].d = 0
    b_Q = binheap(B_G.V)

    for h_i, node in enumerate(f_Q._A):
        node.heap_index = h_i
    for h_i, node in enumerate(b_Q._A):
        node.heap_index = h_i
    
    #print(f_Q)
    #print(b_Q)

    forward_processed = []
    backward_processed = []
    i = 0
    final_weight = 0
    while not f_Q.is_empty() and not b_Q.is_empty():
        #print(f_Q)
        #print(b_Q)
        u = f_Q.remove_minimum()
        v = b_Q.remove_minimum()

        flag = False

        f_queue = []
        b_queue = []
        dist = 0

        print(u.index,v.index)
        if u.index==v.index:
            flag = True
            f_queue.append(u)
            get_predecessors(u,f_queue)
            b_queue.append(v)
            get_predecessors(v,b_queue)

            f_queue.reverse()
            final_weight = f_queue[-1].d + b_queue[0].d
            f_queue.pop()
            break
        
        else:
            for el in backward_processed:
                if u.index == el.index:
                    flag = True
                    get_predecessors(u,f_queue)
                    b_node = [x for x in B_G.V if x.index == u.index][0]
                    b_queue.append(b_node)
                    get_predecessors(b_node,b_queue)
                    dist = u.d + b_node.d
            
            for el in forward_processed:
                if v.index == el.index:
                    flag = True
                    b_queue.append(v)
                    get_predecessors(v,b_queue)
                    f_node = [x for x in F_G.V if x.index == u.index][0]
                    get_predecessors(f_node,f_queue)
                    dist = v.d + f_node.d
            

            if flag is True:
                f_queue.reverse()
                final_weight = dist
                break
        

        forward_processed.append(u)
        backward_processed.append(v)
        
        # this means that we've arrived at the point of the queue
        # in which are located only nodes eliminated by shortcuts
        if u.d == inf and u.pred is None or v.d == inf and v.pred is None:
            break
        
        if i != 0:
            print(f"\nforward node {u.index} with total distance {u.d} from root and predecessor node is {u.pred.index}")
            print(f"backward node {v.index} with total distance {v.d} from root and predecessor node is {v.pred.index}")
        else:
            i += 1
            print(f"The forward path starts from node {u.index}, then we have:")
            print(f"The backward path starts from node {v.index}, then we have:")
        

        for (k,w) in F_G.Adj[u.index]:
            relax(f_Q, u, k, w)
            #if k in backward_processed and u.d + k.d + w < mu:
            #    mu = u.d + k.d + w  
        
        for (k,w) in B_G.Adj[v.index]:
            relax(b_Q, v, k, w)
            #if k in forward_processed and v.d + k.d + w < mu:
            #    mu = v.d + k.d + w
    
    return f_queue+b_queue, final_weight


In [12]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(5,None,None,None)
e = Node(4,None,None,None)
f = Node(3,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,1), Edge(a,c,10), Edge(b,c,1), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]
backward_edges = reverse_edges(edges)

G1 = Weighted_Graph(vertices, edges)
G2 = Weighted_Graph(vertices, backward_edges)

In [13]:
res, weight = bidirect_Dijkstra(G1, G2, a, e)

0 4
The forward path starts from node 0, then we have:
The backward path starts from node 4, then we have:
1 5

forward node 1 with total distance 1 from root and predecessor node is 0
backward node 5 with total distance 1 from root and predecessor node is 4
2 2


In [14]:
for el in res:
    el.print_node()
print(weight)

index: 0, data: 0, predecessor: None
index: 1, data: 1, predecessor: 0
index: 2, data: 3, predecessor: 5
index: 5, data: 1, predecessor: 4
index: 4, data: 0, predecessor: None
5


In [52]:
def E_up(vertices, edges, updated_edges, num_og_edges, source, target, f=compute_shortcut, index=0):
    if index == num_og_edges:
        return updated_edges

    edge = edges[index]
    index += 1
    if edge.v.index < edge.u.index: #and edge.v.index != source.index and edge.v.index != target.index:
        new = f(edge.v, updated_edges, vertices)
        return E_up(vertices, edges, new, num_og_edges, source, target, f, index)
    else:
        return E_up(vertices, edges, updated_edges, num_og_edges, source, target, f, index)    

def E_down(vertices, edges, updated_edges, num_og_edges, source, target, f=compute_shortcut, index=0):
    if index == num_og_edges:
        return updated_edges

    edge = edges[index]
    index += 1
    if edge.v.index > edge.u.index: #and edge.v.index != source.index and edge.v.index != target.index:
        new = f(edge.v, updated_edges, vertices)
        return E_down(vertices, edges, new, num_og_edges, source, target, f, index)
    else:
        return E_down(vertices, edges, updated_edges, num_og_edges, source, target, f, index)

In [48]:
forward_preprocessed_edges = E_up(vertices, edges, updated_edges=edges, num_og_edges=len(edges), f=compute_shortcut, index=0)

TypeError: E_up() missing 2 required positional arguments: 'source' and 'target'

In [17]:
for el in forward_preprocessed_edges:
    print(f"pred: {el.u.index}, succ: {el.v.index}, val: {el.weight}")

pred: 0, succ: 1, val: 1
pred: 0, succ: 2, val: 10
pred: 1, succ: 2, val: 1
pred: 2, succ: 5, val: 2


In [18]:
backward_preprocessed_edges = E_down(vertices, edges, updated_edges=edges, num_og_edges=len(edges), f=compute_shortcut, index=0)

deleted -> 1
pred -> [0]
pred_weight -> [1]
next -> [2, 3]
next_weight -> [1, 15]
deleted -> 2
pred -> [0, 0]
pred_weight -> [10, 2]
next -> [5]
next_weight -> [2]
deleted -> 2
pred -> []
pred_weight -> []
next -> []
next_weight -> []
deleted -> 3
pred -> [4, 0]
pred_weight -> [3, 16]
next -> []
next_weight -> []
deleted -> 5
pred -> [0, 0]
pred_weight -> [12, 4]
next -> [4]
next_weight -> [1]


In [19]:
for el in backward_preprocessed_edges:
    print(f"pred: {el.u.index}, succ: {el.v.index}, val: {el.weight}")

pred: 0, succ: 4, val: 13
pred: 0, succ: 4, val: 5


In [56]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)
g = Node(6,None,None,None)
h = Node(7,None,None,None)

vertices = [a,b,c,d,e,f,g,h]
edges = [Edge(b,c,2), Edge(c,b,1), Edge(c,d,3), Edge(a,e,1), Edge(e,a,3), Edge(h,a,1), Edge(h,g,1), Edge(g,h,1), Edge(d,h,3), Edge(e,f,1), Edge(f,h,1), Edge(a,f,1)]
forward_edges = E_up(vertices, edges, updated_edges=edges, num_og_edges=len(edges), source=c, target=e, f=compute_shortcut, index=0)
print("\n")
backward_edges = E_down(vertices, edges, updated_edges=edges, num_og_edges=len(edges), source=c, target=e, f=compute_shortcut, index=0)
edges.extend(forward_edges)
#edges.append(backward_edges)

G1 = Weighted_Graph(vertices, edges)
G2 = Weighted_Graph(vertices, reverse_edges(edges))

deleted -> 1
pred -> [2]
pred_weight -> [1]
next -> [2]
next_weight -> [2]
deleted -> 0
pred -> [4, 7]
pred_weight -> [3, 1]
next -> [4, 5]
next_weight -> [1, 1]
deleted -> 0
pred -> []
pred_weight -> []
next -> []
next_weight -> []
deleted -> 6
pred -> [7]
pred_weight -> [1]
next -> [7]
next_weight -> [1]


deleted -> 2
pred -> [1]
pred_weight -> [2]
next -> [1, 3]
next_weight -> [1, 3]
deleted -> 3
pred -> [1]
pred_weight -> [5]
next -> [7]
next_weight -> [3]
deleted -> 4
pred -> [0]
pred_weight -> [1]
next -> [0, 5]
next_weight -> [3, 1]
deleted -> 7
pred -> [6, 5, 1]
pred_weight -> [1, 1, 8]
next -> [0, 6]
next_weight -> [1, 1]
deleted -> 7
pred -> []
pred_weight -> []
next -> []
next_weight -> []
deleted -> 5
pred -> [0]
pred_weight -> [1]
next -> [0, 6]
next_weight -> [2, 2]
deleted -> 7
pred -> []
pred_weight -> []
next -> []
next_weight -> []
deleted -> 5
pred -> []
pred_weight -> []
next -> []
next_weight -> []


In [57]:
res, weight = bidirect_Dijkstra(G1,G2,c,e)
for el in res:
    el.print_node()
print(weight)

2 4
The forward path starts from node 2, then we have:
The backward path starts from node 4, then we have:
1 0

forward node 1 with total distance 1 from root and predecessor node is 2
backward node 0 with total distance 1 from root and predecessor node is 4
3 7

forward node 3 with total distance 3 from root and predecessor node is 2
backward node 7 with total distance 2 from root and predecessor node is 4
7 6
index: 2, data: 0, predecessor: None
index: 3, data: 3, predecessor: 2
index: 7, data: 2, predecessor: 4
index: 4, data: 0, predecessor: None
8


In [27]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)
g = Node(6,None,None,None)
h = Node(7,None,None,None)

vertices = [a,b,c,d,e,f,g,h]
edges = [Edge(b,c,2), Edge(c,b,1), Edge(c,d,3), Edge(a,e,1), Edge(e,a,3), Edge(h,a,1), Edge(h,g,1), Edge(g,h,1), Edge(d,h,3), Edge(e,f,1), Edge(f,h,1), Edge(a,f,1)]
backward_edges = reverse_edges(edges)
G1 = Weighted_Graph(vertices, edges)
G2 = Weighted_Graph(vertices, backward_edges)

res, weight = bidirect_Dijkstra(G1,G2,c,e)

2 4
The forward path starts from node 2, then we have:
The backward path starts from node 4, then we have:
1 0

forward node 1 with total distance 1 from root and predecessor node is 2
backward node 0 with total distance 1 from root and predecessor node is 4
3 7

forward node 3 with total distance 3 from root and predecessor node is 2
backward node 7 with total distance 2 from root and predecessor node is 0
7 6


In [22]:
for el in res:
    el.print_node()
print(weight)

index: 2, data: 0, predecessor: None
index: 3, data: 3, predecessor: 2
index: 7, data: 2, predecessor: 0
index: 0, data: 1, predecessor: 4
index: 4, data: 0, predecessor: None
8


In [23]:
new_edges = compute_shortcut(a, edges, vertices)
G1 = Weighted_Graph(vertices, new_edges)
G2 = Weighted_Graph(vertices, reverse_edges(new_edges))
res, weight = bidirect_Dijkstra(G1,G2,c,e)

deleted -> 0
pred -> [4, 7]
pred_weight -> [3, 1]
next -> [4, 5]
next_weight -> [1, 1]
2 4
The forward path starts from node 2, then we have:
The backward path starts from node 4, then we have:
1 7

forward node 1 with total distance 1 from root and predecessor node is 2
backward node 7 with total distance 2 from root and predecessor node is 4
3 6

forward node 3 with total distance 3 from root and predecessor node is 2
backward node 6 with total distance 3 from root and predecessor node is 7
7 5


In [24]:
for el in res:
    el.print_node()
print(weight)

index: 2, data: 0, predecessor: None
index: 3, data: 3, predecessor: 2
index: 7, data: 2, predecessor: 4
index: 4, data: 0, predecessor: None
8
